## GridWorld
Ph.D Leonarod A, Espinosa, M.Sc Andrej Scherbakov-Parland, BIT Kristoffer Kuvaja Adolfsson

### Bibliography:

* Sutton, Richard S., and Andrew G. Barto. Reinforcement learning: An introduction. MIT press, 2018.
http://incompleteideas.net/book/bookdraft2017nov5.pdf  (chapter 4)

In [ ]:
# imports
import numpy as np
from IPython.display import clear_output
import time

In [ ]:
class GridWorld(object):
    """ Gridworld defined by m x n matrix with
    terminal states at top left corner and bottom right corner.
    State transitions are deterministic; attempting to move
    off the grid leaves the state unchanged, and rewards are -1 on
    each step. 

    In this implementation we model the environment like a game
    where an agent moves around.
    """
    def __init__(self, m, n):
        self.m = m
        self.n = n
        self.grid = np.zeros((m,n))
        self.stateSpace = [i+1 for i in range(self.m*self.n-2)]
        self.stateSpacePlus = [i for i in range(self.m*self.n)]        
        self.actionSpace = {'up': -self.m, 'down': self.m, 
                            'left': -1, 'right': 1}
        self.p = self.initP() # probability functions
        self.agentPosition = np.random.choice(self.stateSpace)
        x, y = self.getAgentRowAndColumn() 
        self.grid[x][y] = 1

    def getAgentRowAndColumn(self):
        x = self.agentPosition // self.m
        y = self.agentPosition % self.n
        return x, y

    def setState(self, state):
        x, y = self.getAgentRowAndColumn() 
        self.grid[x][y] = 0            
        self.agentPosition = state        
        x, y = self.getAgentRowAndColumn() 
        self.grid[x][y] = 1  

    def offGridMove(self, newState, oldState):
        # if we move into a row not in the grid
        if newState not in self.stateSpacePlus:
            return True
        # if we're trying to wrap around to next row
        elif oldState % self.m == 0 and newState  % self.m == self.m - 1:
            return True
        elif oldState % self.m == self.m - 1 and newState % self.m == 0:
            return True
        else:
            return False        
    def step(self, action):        
        resultingState = self.agentPosition + self.actionSpace[action]
        if not self.offGridMove(resultingState, self.agentPosition):
            self.setState(resultingState)
            return resultingState, -1, self.isTerminalState(resultingState), None
        else:
            return self.agentPosition, -1, self.isTerminalState(self.agentPosition), None
    
    def isTerminalState(self, state):
        return state in self.stateSpacePlus and state not in self.stateSpace      

    def initP(self):
        """ construct state transition probabilities for
        use in value function. P(s', r|s, a) is a dictionary
        with keys corresponding to the functional arguments.
        values are either 1 or 0.
        Translations that take agent off grid leave the state unchanged.
        (s', r|s, a)
        (1, -1|1, 'up') = 1
        (1, -1|2, 'left') = 1
        (1, -1|3, 'left') = 0        
        """
        P = {}
        for state in self.stateSpace:
            for action in self.actionSpace:
                resultingState = state + self.actionSpace[action]
                key = (state, -1, state, action) if self.offGridMove(resultingState, state) \
                                                 else (resultingState, -1, state, action)
                P[key] = 1
        return P

    def render(self):
        print('------------------------------------------')
        for row in self.grid:
            for col in row:
                if col == 0:
                    print('-', end='\t')
                elif col == 1:
                    print('X', end='\t')
            print('\n')
        print('------------------------------------------')

## Del 1 - Utvärdera policyn

In [ ]:
def evaluatePolicy(grid, V, policy, GAMMA, THETA):
    # policy evaluation for the random choice in gridworld
    converged = False
    iterations = 0 # <---
    while not converged:
        DELTA = 0
        for state in grid.stateSpace:
            oldV = V[state]
            total = 0
            weight = 1 / len(policy[state])           
            for action in policy[state]:
                grid.setState(state)
                newState, reward, _, _ = grid.step(action)
                key = (newState, reward, state, action)
                total += weight*grid.p[key]*(reward+GAMMA*V[newState])
            V[state] = total
            DELTA = max(DELTA, np.abs(oldV-V[state]))
            converged = True if DELTA < THETA else False
        iterations = iterations + 1 # <---
    print('iterations: ', iterations) # <----
    return V

In [ ]:
def printV(V, grid):
    for idx, row in enumerate(grid.grid):
        for idy, _ in enumerate(row):            
            state = grid.m * idx + idy 
            print('%.2f' % V[state], end='\t')
        print('\n')
    print('--------------------')

In [ ]:
def get_policy(m, n, x, y):
    grid = GridWorld(m,n) # Rutnät m x n
    THETA = x # Konvergensvillkor
    GAMMA = y
    
    # initialize V(s)
    V = {}
    for state in grid.stateSpacePlus:        
        V[state] = 0
    
    policy = {}
    for state in grid.stateSpace:
        policy[state] = [key for key in grid.actionSpace.keys()]

    
    V = evaluatePolicy(grid, V, policy, GAMMA, THETA)
    printV(V, grid)   

### Hands-on del 1
- Implementera utvärderingspolicyfunktionen (evaluatePolicy) i en python-notebook.
- Testa koden för 4x4, 8x8 och 10x10 rutnät.
    - Vad händer med antalet iterationer som krävs för att konvergera?
    - Vad händer med tiden?
- Vad händer om gamma är lika med 0,5, 0,9 för 4x4 och 8x8? Summera och  gör slutsatser.

## Del 2 - Förbättra policyn

In [ ]:
def improvePolicy(grid, V, policy, GAMMA):
    stable = True
    newPolicy = {}
    for state in grid.stateSpace:       
        oldActions = policy[state]                
        value = []
        newAction = []
        for action in policy[state]:
            grid.setState(state)
            weight = 1 / len(policy[state])
            newState, reward, _, _ = grid.step(action)
            key = (newState, reward, state, action)
            value.append(np.round(weight*grid.p[key]*(reward+GAMMA*V[newState]), 2))
            newAction.append(action)
        value = np.array(value)        
        best = np.where(value == value.max())[0]        
        bestActions = [newAction[item] for item in best] 
        newPolicy[state] = bestActions

        if oldActions != bestActions:
            stable = False
        
    return stable, newPolicy

In [ ]:
def get_policy_iteration(m, n, x, y):

    grid = GridWorld(m,n)
    THETA = x
    GAMMA = y
    
    # initialize V(s)

    V = {}
    for state in grid.stateSpacePlus:        
        V[state] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = [key for key in grid.actionSpace.keys()]

    # main loop for policy improvement
    stable = False
    while not stable:
        V = evaluatePolicy(grid, V, policy, GAMMA, THETA)
        stable, policy = improvePolicy(grid, V, policy, GAMMA)       
        V = evaluatePolicy(grid, V, policy, GAMMA, THETA)

        printV(V, grid) 
        time.sleep(2)
        clear_output(wait=True)

    printV(V,grid)

    for state in policy:
        print(state,policy[state])

### Hands-on del 2.

- Implementera den förbättrade policyfunktionen (improvePolicy).
- Testa koden för 4x4, 8x8 och 10x10 rutnät.
    - Vad händer med antalet iterationer som krävs för att konvergera?
    - Vad händer med tiden?
- Hur olika är policyerna?

## Del 3 - Värde iteration

In [ ]:
def iterateValues(grid, V, policy, GAMMA, THETA):
    converged = False
    iterations = 0
    while not converged:
        DELTA = 0
        for state in grid.stateSpace:
            oldV = V[state]
            newV = []            
            for action in grid.actionSpace:
                grid.setState(state)
                newState, reward, _, _ = grid.step(action)
                key = (newState, reward, state, action) 
                newV.append(grid.p[key]*(reward+GAMMA*V[newState]))                
            newV = np.array(newV)
            bestV = np.where(newV == newV.max())[0]
            bestState = np.random.choice(bestV)
            V[state] = newV[bestState]
            DELTA = max(DELTA, np.abs(oldV-V[state]))
            converged = True if DELTA < THETA else False
        iterations = iterations + 1
    print('iterations: ', iterations)

    for state in grid.stateSpace:
        newValues = []
        actions = []
        for action in grid.actionSpace:
            grid.setState(state)
            newState, reward, _, _ = grid.step(action)
            key = (newState, reward, state, action)
            newValues.append(grid.p[key]*(reward+GAMMA*V[newState]))
            actions.append(action)
        newValues = np.array(newValues)
        bestActionIDX = np.where(newValues == newValues.max())[0]
        bestActions = actions[np.random.choice(bestActionIDX)]
        bestActions = actions[bestActionIDX[0]]
        policy[state] = bestActions

    return V, policy

In [ ]:
def get_value_iteration(m, n, x, y):

    grid = GridWorld(m,n)
    THETA = x
    GAMMA = y

    # initialize V(s)
    V = {}
    for state in grid.stateSpacePlus:        
        V[state] = 0

    policy = {}
    for state in grid.stateSpace:
        policy[state] = [key for key in grid.actionSpace.keys()]

    for i in range(2):
        V, policy = iterateValues(grid, V, policy, GAMMA, THETA)
        printV(V, grid)   
        time.sleep(2)
        clear_output(wait=True)
    # Final   

    printV(V, grid) 
    print()
    for state in policy:
        print(state, policy[state])

### Hands-on del 3.

- Implementera ännu en förbättrad policyfunktion (iterateValues).
- Testa koden för 4x4, 8x8 och 10x10 rutnät.
    - Vad händer med antalet iterationer som krävs för att konvergera?
    - Vad händer med tiden?
- Hur olika är policyerna för gamma=0.5,0.9 i ett 8x8-rutnät?